In [10]:
import seaborn as sns
import pandas as pd
from sklearn.datasets import load_breast_cancer
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier, GBTClassifier

In [2]:
spark = SparkSession.builder.appName('clasificacion_binaria').getOrCreate()
data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['label'] = data.target
df.columns = [col.replace(' ', '_') for col in df.columns]
df.head(5)

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,mean_compactness,mean_concavity,mean_concave_points,mean_symmetry,mean_fractal_dimension,...,worst_texture,worst_perimeter,worst_area,worst_smoothness,worst_compactness,worst_concavity,worst_concave_points,worst_symmetry,worst_fractal_dimension,label
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [3]:
df = spark.createDataFrame(df)
feature_cols = [col for col in df.columns if col != 'label']
feature_cols

['mean_radius',
 'mean_texture',
 'mean_perimeter',
 'mean_area',
 'mean_smoothness',
 'mean_compactness',
 'mean_concavity',
 'mean_concave_points',
 'mean_symmetry',
 'mean_fractal_dimension',
 'radius_error',
 'texture_error',
 'perimeter_error',
 'area_error',
 'smoothness_error',
 'compactness_error',
 'concavity_error',
 'concave_points_error',
 'symmetry_error',
 'fractal_dimension_error',
 'worst_radius',
 'worst_texture',
 'worst_perimeter',
 'worst_area',
 'worst_smoothness',
 'worst_compactness',
 'worst_concavity',
 'worst_concave_points',
 'worst_symmetry',
 'worst_fractal_dimension']

In [7]:
assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol='features'
)
df_assembled = assembler.transform(df).select('features', 'label')
df_assembled.show(2)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[17.99,10.38,122....|    0|
|[20.57,17.77,132....|    0|
+--------------------+-----+
only showing top 2 rows



In [8]:
#particionamiento de datos
df_train, df_test = df_assembled.randomSplit([0.8, 0.2], seed=42)


In [11]:
lr = LogisticRegression()
model = lr.fit(df_train)
df_pred = model.transform(df_test)
df_pred.show(4)

+--------------------+-----+--------------------+-----------+----------+
|            features|label|       rawPrediction|probability|prediction|
+--------------------+-----+--------------------+-----------+----------+
|[8.618,11.79,54.3...|    1|[-1633.8350728041...|  [0.0,1.0]|       1.0|
|[9.504,12.44,60.3...|    1|[-1684.3539200749...|  [0.0,1.0]|       1.0|
|[10.95,21.35,71.9...|    0|[873.702082903982...|  [1.0,0.0]|       0.0|
|[11.84,18.7,77.93...|    0|[2517.67943875863...|  [1.0,0.0]|       0.0|
+--------------------+-----+--------------------+-----------+----------+
only showing top 4 rows



In [13]:
evaluator_auc = BinaryClassificationEvaluator(metricName='areaUnderROC')
evaluator_accuracy = MulticlassClassificationEvaluator(metricName='accuracy')
evaluator_f1 = MulticlassClassificationEvaluator(metricName='f1')
evaluator_precision = MulticlassClassificationEvaluator(metricName='weightedPrecision')
evaluator_recall = MulticlassClassificationEvaluator(metricName='weightedRecall')

print('auc', evaluator_auc.evaluate(df_pred))
print('accuracy', evaluator_accuracy.evaluate(df_pred))
print('f1', evaluator_f1.evaluate(df_pred))
print('precision', evaluator_precision.evaluate(df_pred))
print('recall', evaluator_recall.evaluate(df_pred))


auc 0.9932026143790849
accuracy 0.9523809523809523
f1 0.9523809523809523
precision 0.9523809523809523
recall 0.9523809523809523


In [ ]:
rfc = RandomForestClassifier()
model = rfc.fit(df_train)
df_pred = model.transform(df_test)
df_pred.show(4)
print('auc', evaluator_auc.evaluate(df_pred))
print('accuracy', evaluator_accuracy.evaluate(df_pred))
print('f1', evaluator_f1.evaluate(df_pred))
print('precision', evaluator_precision.evaluate(df_pred))
print('recall', evaluator_recall.evaluate(df_pred))


+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[8.618,11.79,54.3...|    1|[0.11952245562896...|[0.00597612278144...|       1.0|
|[9.504,12.44,60.3...|    1|[0.13422833798191...|[0.00671141689909...|       1.0|
|[10.95,21.35,71.9...|    0|[3.89525941443441...|[0.19476297072172...|       1.0|
|[11.84,18.7,77.93...|    0|[17.6361299187386...|[0.88180649593693...|       0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 4 rows

auc 0.9979084967320262
accuracy 0.9682539682539683
f1 0.9680183761816414
precision 0.9698613622664256
recall 0.9682539682539683


In [16]:
tree = DecisionTreeClassifier()
model = tree.fit(df_train)
df_pred = model.transform(df_test)
df_pred.show(4)
print('auc', evaluator_auc.evaluate(df_pred))
print('accuracy', evaluator_accuracy.evaluate(df_pred))
print('f1', evaluator_f1.evaluate(df_pred))
print('precision', evaluator_precision.evaluate(df_pred))
print('recall', evaluator_recall.evaluate(df_pred))


+--------------------+-----+-------------+--------------------+----------+
|            features|label|rawPrediction|         probability|prediction|
+--------------------+-----+-------------+--------------------+----------+
|[8.618,11.79,54.3...|    1|  [0.0,229.0]|           [0.0,1.0]|       1.0|
|[9.504,12.44,60.3...|    1|  [0.0,229.0]|           [0.0,1.0]|       1.0|
|[10.95,21.35,71.9...|    0|    [3.0,0.0]|           [1.0,0.0]|       0.0|
|[11.84,18.7,77.93...|    0|  [132.0,1.0]|[0.99248120300751...|       0.0|
+--------------------+-----+-------------+--------------------+----------+
only showing top 4 rows

auc 0.9351633986928104
accuracy 0.9365079365079365
f1 0.9366894366894365
precision 0.9373530751147707
recall 0.9365079365079365


In [17]:
gbt = GBTClassifier()
model = gbt.fit(df_train)
df_pred = model.transform(df_test)
df_pred.show(4)
print('auc', evaluator_auc.evaluate(df_pred))
print('accuracy', evaluator_accuracy.evaluate(df_pred))
print('f1', evaluator_f1.evaluate(df_pred))
print('precision', evaluator_precision.evaluate(df_pred))
print('recall', evaluator_recall.evaluate(df_pred))


+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[8.618,11.79,54.3...|    1|[-1.5467073518361...|[0.04337971073112...|       1.0|
|[9.504,12.44,60.3...|    1|[-1.5517204944636...|[0.04296554045975...|       1.0|
|[10.95,21.35,71.9...|    0|[0.67098509640030...|[0.79281375222118...|       0.0|
|[11.84,18.7,77.93...|    0|[1.41140714026416...|[0.94389628602594...|       0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 4 rows

auc 0.9898039215686275
accuracy 0.9444444444444444
f1 0.944671201814059
precision 0.9459876543209877
recall 0.9444444444444444
